In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity



global representation_movie
df_genre = pd.read_csv('data/movies_w_imgurl.csv', usecols = ['movieId','genres'])

# set of genre
genres_list = []
for line in df_genre['genres'].tolist():
    genres_list.extend(line.split('|'))
genres_list = set(genres_list)
genre_count = { genre:0 for genre in iter(genres_list)}  

# count genre 
for line in df_genre['genres'].tolist(): 
    items = line.split('|')
    for item in items:
        genre_count[item]+=1

# check total count 
movie_ids = df_genre.movieId.unique()
total_count =  movie_ids.shape[0]



# create IDF for genre_count 
TF = 1
for key in genre_count.keys():
    IDF = np.log10(total_count / genre_count[key])
    genre_count[key] = TF * IDF

columns = list(genre_count.keys())

columns= sorted(columns)


representation_movie = pd.DataFrame(index=sorted(movie_ids), columns=columns)




for index,row in df_genre.iterrows(): 
    items = row['genres'].split('|')
    for genre in genres_list:
        if genre in items:
            representation_movie.loc[row['movieId']][genre] = genre_count[genre]


representation_movie= representation_movie.fillna(0)


In [2]:
global representation_tag
df_tag = pd.read_csv('data/tags.csv', usecols=['userId','movieId','tag','timestamp'])


### check total count 
uniq_movie_ids = df_tag.movieId.unique()
uniq_total_count = uniq_movie_ids.shape[0]

# calculate IDF for tag  
### count tags

tag_count = dict()
for index, row in df_tag.iterrows():
    tags= [item.strip() for item in row['tag'].split(',')]
    for tag in tags:
        tag_count.setdefault(tag, 0)
        tag_count[tag]+=1

tag_list = list(tag_count.keys())         
### calculate IDF for tag  
IDFs = dict()
for tag in tag_list:
    IDFs[tag] = np.log10(uniq_total_count / tag_count[tag])
    


representation_tag = pd.DataFrame(index=sorted(movie_ids), columns=sorted(tag_list))
representation_tag=representation_tag.fillna(0.0)

n_d = dict()
rp_tag = dict()
for index, row in df_tag.iterrows():
        
    tags= [item.strip() for item in row['tag'].split(',')]
    movieId = row['movieId']
    n_d.setdefault(movieId,0)
   
    n_d[movieId] += len(tags) 
    rp_tag.setdefault(movieId,{})
    
    for tag in tags:
        rp_tag[movieId].setdefault(tag, 0)
        rp_tag[movieId][tag] += 1
    

    
    

In [3]:
# rp_tag[1210]

In [4]:

for movieId in uniq_movie_ids: 
    for tag in rp_tag[movieId].keys():
        n_dt = rp_tag[movieId][tag]
        TFIDF =  (n_dt/n_d[movieId]) * IDFs[tag]
       
        representation_tag.loc[movieId][tag] = TFIDF 

#         if movieId == 1210:
#             print (rp_tag[movieId].keys())
#             print(tag, representation_tag.loc[movieId][tag])
           
        

In [5]:
# rp_tag[1210]
# n_d[1210]

In [6]:
# print(representation_movie.shape)
# print(representation_tag.shape)
representation = pd.concat([representation_movie, representation_tag], axis=1).fillna(0)
# representation.shape

In [7]:
# task 3
similarity = cosine_similarity(representation)
similarity = pd.DataFrame(similarity, index=sorted(movie_ids), columns=sorted(movie_ids))
# similarity

In [60]:

def read_user_id():
    with open('input.txt', 'r') as f:
        return [l.strip() for l in  f.readlines()]

def write_output(prediction):
    with open('output.txt', 'w') as f:
        for pred in prediction:
            for row in pred:
                f.write(row + "\n")

def do(ids):
    string_results = []
    for uid in ids:
        string_results.append(get_top(int(uid)))
    return string_results


global df_rate
df_rate = pd.read_csv('data/ratings.csv', usecols=['userId','movieId','rating'])



def get_top(uid, top=30):
    rated_movies = df_rate[df_rate.userId ==uid]['movieId'].to_frame()
    #print(rated_movies.movieId)
 
#     user_sim = movie_movie_cosine_similarity.loc[user_movie_ratings[user_id].movieId]
   
    user_sim = similarity.loc[rated_movies.movieId]
    # print(user_sim.shape
    
    user_rating = df_rate[df_rate.userId == uid]['rating'].to_frame()
#     print(user_rating.shape)
    sim_sum = user_sim.sum(axis=0).to_frame()
    prediction = np.matmul(user_sim.T.to_numpy(), user_rating.to_numpy()) / (sim_sum.to_numpy() + 1)   
    prediction = prediction.round(4)
    
    prediction =  pd.DataFrame(prediction, index=sorted(movie_ids), columns=['prediction'])
#     print(prediction)
    prediction['movieId']=sorted(movie_ids)
    prediction = prediction.sort_values(by=['prediction','movieId'], ascending=[False,True]).head(top)
 
    result=[]
    for index, line in prediction.iterrows():
        
        line=','.join([str(uid), str(index), str(line['prediction'])]) 
#         print(line)
        result.append(line)
    # prediction['movieId'] = prediction.index

    return result 
    




In [61]:
# task 4 recommending

user_ids = read_user_id()
result = do(user_ids)
write_output(result)